# Домашнее задание 1 (50 баллов)
v4

В этом домашнем задании вы познакомитесь с основами NLP, научитесь обрабатывать тексты.

Будет несколько теоретических вопросов, обозначенных символом 🤔.

Блоки, где необходимо написать свой код, обозначены через YOUR_CODE_HERE. Код, который уже написан, менять не нужно. Блоки в разных частях задания зависят друг от друга, поэтому не вносите больших изменений, чтобы ничего не сломалось.

In [ ]:
#!pip install gensim nltk torch tqdm seqeval

In [ ]:
from typing import List, Dict

In [ ]:
YOUR_CODE_HERE = None  # заглушка, здесь ничего не трогайте

## Токенизация (5 баллов)

Токенизация - это процесс преобразования текста в набор токенов.
Наивная реализация разбивает текст по пробелам. Более умные реализации учитывают пунктуацию.

Научимся работать с токенизацией NLTK, где уже реализована работа с пунктуацией.

https://www.nltk.org/

In [ ]:
import nltk

# https://www.nltk.org/nltk_data/
nltk.download("punkt")
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
def tokenize(text: str, language: str = "english", lower: bool = False) -> List[str]:
    # YOUR_CODE_HERE
    if lower:  text = text.lower()
    return nltk.word_tokenize(text)



assert tokenize("") == []
assert tokenize("Hello, world!") == ["Hello", ",", "world", "!"]
assert tokenize("EU rejects German call to boycott British lamb.") == ["EU", "rejects", "German", "call", "to", "boycott", "British", "lamb", "."]

In [ ]:
def split_sentences(text: str, language: str = "english", lower: bool = False,
                    separators = '.!?') -> List[str]:
    # YOUR_CODE_HERE
    if lower:  text = text.lower()

    sentences,  sentence = [],  ""

    # Идем по словам текста
    for char in text:
        sentence += char
        # проверяем по набору разделителей
        if char in separators:
            sentences.append(sentence.strip())  # обрезаеи и добавляем в список
            sentence = ""  # очищаем

    # Добавляем оставшееся в лист
    if sentence:
        sentences.append(sentence.strip())

    return sentences



assert split_sentences("") == []
assert split_sentences("Hello, world!") == ["Hello, world!"]
assert split_sentences("Hello, world! I love Python!") == ["Hello, world!", "I love Python!"]

## Векторизация (5 баллов)

Чтобы работать с текстами, нужно преобразовать токены (слова) в векторы.
Для этого сначала необходимо составить словарь и сопоставить каждому слову уникальный индекс.
В образовательных целях реализуем этот словарь с нуля.

In [ ]:
from collections import defaultdict, Counter

In [ ]:
import re
from collections import Counter, defaultdict
from typing import List

class Vocabulary:
    def __init__(self, texts: List[str], language: str = "english", min_count: int = 1, lower: bool = False):
        """
        Создание словаря.

        :param texts: коллекция текстов
        :param language: язык текстов
        :param min_count: минимальная частота слова для попадания в словарь
        """
        self.language = language
        self.lower = lower
        self.min_count = min_count
        self.word2idx = {"<PAD>": 0, "<SOS>": 1, "<EOS>": 2, "<UNK>": 3}
        self.idx2word = {0: "<PAD>", 1: "<SOS>", 2: "<EOS>", 3: "<UNK>"}

        # https://docs-python.ru/standart-library/modul-collections-python/klass-counter-modulja-collections/
        self.word2count = Counter()

        self._build_vocabulary(texts)

    def _build_vocabulary(self, texts: List[str]):
        """
        Создание словаря на основе текстов.
        """
        for text in texts:
            if self.lower:
                text = text.lower()
            words = tokenize(text)  # токенезируем текст
            self.word2count.update(words)  # пополняем элементы счетчика Counter

        for word, count in self.word2count.items():
            if count >= self.min_count and word not in self.word2idx:
                idx = len(self.word2idx)
                self.word2idx[word] = idx  # Назначаем новый индекс каждому слову в словаре word2idx
                self.idx2word[idx] = word  # Тоже для ивертированого словаря idx2word

    def encode_word(self, text: str) -> int:
        """
        Кодирование слова в индекс.
        """
        if self.lower:
            text = text.lower()
        return self.word2idx.get(text, self.word2idx["<UNK>"])

    def encode(self, text: str) -> List[int]:
        """
        Кодирование текста в набор индексов.

        :param text: текст
        :return: набор индексов токенов
        """
        if self.lower:
            text = text.lower()
        words = tokenize(text)  # токенезируем текст
        # возвращаеи индексы
        return [self.word2idx.get(word, self.word2idx["<UNK>"]) for word in words]

    def decode(self, input_ids: List[int]) -> str:
        """
        Декодирование набора индексов в текст.

        :param input_ids: набор индексов токенов
        :return: текст
        """
        return " ".join([self.idx2word[idx] for idx in input_ids if idx != self.word2idx["<PAD>"]])

    def __len__(self):
        return len(self.word2idx)

    def __contains__(self, item):
        return item in self.word2idx

    def __iter__(self):
        return iter(self.word2idx)

    def __str__(self):
        return str(self.word2idx)



In [ ]:
vocab = Vocabulary(["Hello, world!", "I love Python!"], min_count=1, lower=True)
encoded = vocab.encode("Hello, Python! I love you")
assert vocab.decode(encoded) == "hello , python ! i love <UNK>"

## $n$-граммные языковые модели (10 баллов)

Напишем собственную $n$-граммную модель.

In [ ]:
import random
class NGramLanguageModel:
    def __init__(self, n: int, vocabulary: Vocabulary, texts: List[str], use_max : bool = True):
        """
        Создание n-граммной языковой модели.

        :param n: порядок n-грамм
        :param vocabulary: словарь
        :param use_max: как выбирать сл-й токен
        """
        assert n >= 2
        self.n = n
        self.vocabulary = vocabulary
        # https://docs-python.ru/standart-library/modul-collections-python/klass-counter-modulja-collections/
        self.frequencies = defaultdict(lambda: Counter())  # частота n-грамм
        self.frequencies_of_prefixes = Counter()  # сумма частот n-грамм для префиксов

        self.use_max = use_max # как генерировать
        self._build_model(texts)


    def _build_model(self, texts: List[str]):
        """
        Построение модели на основе текстов.
        _____
        Модель строит n-граммы из обучающих текстов.
        Для каждой n-граммы она записывает, как часто каждый токен следует
        за заданным префиксом (n-1 предшествующих токенов).
        """
        for text in texts:
            # Добавляем <EOS> как окончание
            tokens = self.vocabulary.encode(text) + [self.vocabulary.word2idx["<EOS>"]]
            for i in range(len(tokens)):  # Движемся по токенам

                # YOUR_CODE_HERE
                if i + self.n - 1 < len(tokens):  # Проверяем наличие n-1 токена
                  prefix = tuple(tokens[i:i + self.n - 1])  # Получаем префикс из n-1 предыдущх токенов
                  token = tokens[i + self.n - 1]  # Последующи токен (n-th)
                  self.frequencies[prefix][token] += 1  # увеличиваем частоту n-грамм
                  self.frequencies_of_prefixes[prefix] += 1  # увеличиваем частоту префикса

    def _get_probability(self, prefix: List[int], token: int) -> float:
        """
        Вероятность токена по префиксу.
        ______
        вычисляется вероятность токена, следующего за заданным префиксом,
        как отношение количества конкретной n-граммы к общему количеству префикса.
        """
        # YOUR_CODE_HERE
        prefix_tuple = tuple(prefix)
        if self.frequencies_of_prefixes[prefix_tuple] == 0:
            return 0.0
        # считаем отношение количества встречания префикса перед токеном
        # к колучеству префиксов
        return self.frequencies[prefix_tuple][token] / self.frequencies_of_prefixes[prefix_tuple]


    def generate_next_token(self, prefix: List[int]) -> int:
        """
        Генерация следующего токена по префиксу.
        ____
        Функция предсказывает следующий токен на основе заданного префикса.
        Используетсяrandom.choices для выбора следующего токена,
        взвешенного по вероятностям или по макс, рассчитанным _get_probability.

        :param prefix: префикс
        :return: следующий токен
        """
        # YOUR_CODE_HERE
        prefix_tuple = tuple(prefix)
        # проверяем есть ли префикс в словаре часотности префиксов
        if prefix_tuple not in self.frequencies:
            # возвращаем индекс спецтокена для неизвестного
            return self.vocabulary.word2idx["<UNK>"]

        # Получаем все возможные следующие токены next_tokens и их вероятности probabilities
        next_tokens = list(self.frequencies[prefix_tuple].keys())
        probabilities = [self._get_probability(prefix, token) for token in next_tokens]

        # Выбрираем следующий токен на основе вероятностей
        if self.use_max:
            next_token = next_tokens[probabilities.index(max(probabilities))]
        else:
           next_token = random.choices(next_tokens, weights=probabilities)[0]

        return next_token

    def autocomplete(self, text: str, max_len: int = 32) -> str:
        """
        Автоматическое завершение текста.
        ___
        Функция генерирует токены на основе последних n-1 токенов (префикса)
        до тех пор, пока не достигнет максимальной длины или не встретит токен <EOS>.

        :param text: текст
        :param max_len: максимальная длина текста
        :return: завершенный текст
        """
        tokens = self.vocabulary.encode(text)
        assert tokens

        # YOUR_CODE_HERE
        while len(tokens) < max_len and tokens[-1] != self.vocabulary.word2idx["<EOS>"]:
            prefix = tokens[-(self.n - 1):]  # Получаем последние n-1 токенов в качестве префикса
            next_token = self.generate_next_token(prefix)
            tokens.append(next_token)

        return self.vocabulary.decode(tokens)

In [ ]:
texts = ["Hello, world!", "I love Python!", "Hello, Python"]
vocab = Vocabulary(["Hello, world!", "I love Python!"], min_count=1, lower=True)
print(vocab)
ngram_lm = NGramLanguageModel(2, vocab, texts)
assert ngram_lm.autocomplete("Hello, Python", max_len=10) == "hello , python ! <EOS>"

{'<PAD>': 0, '<SOS>': 1, '<EOS>': 2, '<UNK>': 3, 'hello': 4, ',': 5, 'world': 6, '!': 7, 'i': 8, 'love': 9, 'python': 10}


🤔 Можно ли использовать $n$-граммную языковую модель, когда длина префикса меньше, чем $n-1$? Если да, то как? Если нет, то почему?

In [ ]:
# Можно добавляеть, имеющийся уже спецтокеn <SOS> если получаем длину префикса менее нужной
# см обновленный класс с обновленным методом _build_model

class NGramLanguageModel_UP(NGramLanguageModel):
    """
    Унаследуем класс и корректируем метод _build_model
    для добавления спецтокена <SOS>
    """

    def _build_model(self, texts: List[str]):
        """
        Построение модели на основе текстов.
        _____
        Модель строит n-граммы из обучающих текстов.
        Для каждой n-граммы она записывает, как часто каждый токен следует
        за заданным префиксом (n-1 предшествующих токенов).
        """
        for text in texts:
            # Добавляем <EOS> как окончание
            tokens = self.vocabulary.encode(text) + [self.vocabulary.word2idx["<EOS>"]]
            for i in range(len(tokens)):  # Движемся по токенам

                # Если предыдущих токенов менее чем n
                if i + self.n - 1 >= len(tokens):
                  print("add <SOS>")
                  # добавляем спецьокен <SOS>
                  tokens+=[self.vocabulary.word2idx["<SOS>"]]
                prefix = tuple(tokens[i:i + self.n - 1])  # Получаем префикс из n-1 предыдущх токенов
                token = tokens[i + self.n - 1]  # Последующи токен (n-th)
                self.frequencies[prefix][token] += 1  # увеличиваем частоту n-грамм
                self.frequencies_of_prefixes[prefix] += 1  # увеличиваем частоту префикса


In [ ]:
texts = ["Hello, world!", "I love Python!", "Hello, Python"]
vocab = Vocabulary(["Hello, world!", "I love Python!"], min_count=1, lower=True)
print(vocab)
ngram_lm = NGramLanguageModel_UP(2, vocab, texts)
assert ngram_lm.autocomplete("Hello, Python", max_len=10) == "hello , python ! <EOS>"

{'<PAD>': 0, '<SOS>': 1, '<EOS>': 2, '<UNK>': 3, 'hello': 4, ',': 5, 'world': 6, '!': 7, 'i': 8, 'love': 9, 'python': 10}
add <SOS>
add <SOS>
add <SOS>


🤔 Что произойдет, если в $n$-граммной языковой модели взять достаточно большое $n$?

In [ ]:
texts = ["Hello, world!", "I love Python!", "Hello, Python"]
vocab = Vocabulary(["Hello, world!", "I love Python!"], min_count=1, lower=True)
print(vocab)
ngram_lm = NGramLanguageModel(5, vocab, texts)
print(ngram_lm.autocomplete("Hello, Python", max_len=10))

{'<PAD>': 0, '<SOS>': 1, '<EOS>': 2, '<UNK>': 3, 'hello': 4, ',': 5, 'world': 6, '!': 7, 'i': 8, 'love': 9, 'python': 10}
hello , python <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK>


Для хорошего обучения n-граммной модели с большим 𝑛
n требуется очень большой корпус текстов. Если текстов недостаточно, модель сталкивается с проблемой разреженности и начинает часто предсказывать редко встречающиеся или некорректные слова.

Когда n в n-граммной модели становится слишком большим:

- Преимущества: Модель получает более широкий контекст, что может улучшить точность в очень специфичных сценариях.
- Недостатки:
   - Разреженность данных: Для большого 𝑛 модель может не находить подходящих n-грамм в текстах.
   - Переобучение: Модель может запоминать конкретные последовательности и плохо обобщать на новые данные.
   - Высокие затраты на память и вычисления.


## Word2Vec (5 баллов)

Познакомимся с моделью Word2Vec. Для русского языка есть обученные модели.
https://rusvectores.org/ru/models/

Модели для английского языка: https://github.com/piskvorky/gensim-data

Научимся доставать эмбеддинги слов из обученных моделей.

In [ ]:
from gensim.models import KeyedVectors
import gensim.downloader as api

In [ ]:
info = api.info()  # show info about available models/datasets
info

{'corpora': {'semeval-2016-2017-task3-subtaskBC': {'num_records': -1,
   'record_format': 'dict',
   'file_size': 6344358,
   'reader_code': 'https://github.com/RaRe-Technologies/gensim-data/releases/download/semeval-2016-2017-task3-subtaskB-eng/__init__.py',
   'license': 'All files released for the task are free for general research use',
   'fields': {'2016-train': ['...'],
    '2016-dev': ['...'],
    '2017-test': ['...'],
    '2016-test': ['...']},
   'description': 'SemEval 2016 / 2017 Task 3 Subtask B and C datasets contain train+development (317 original questions, 3,169 related questions, and 31,690 comments), and test datasets in English. The description of the tasks and the collected data is given in sections 3 and 4.1 of the task paper http://alt.qcri.org/semeval2016/task3/data/uploads/semeval2016-task3-report.pdf linked in section “Papers” of https://github.com/RaRe-Technologies/gensim-data/issues/18.',
   'checksum': '701ea67acd82e75f95e1d8e62fb0ad29',
   'file_name': 'se

In [ ]:
# вы можете выбрать любую модель из gensim, например fasstext
w2v_model = api.load("glove-twitter-25")
#w2v_model = KeyedVectors.load_word2vec_format('./gensim/glove-twitter-25.gz')

[==================================================] 100.0% 104.8/104.8MB downloaded


In [ ]:
print(w2v_model["potato"])

[-0.41654   -0.56071    0.72333    1.0435     0.0098203  0.46871
  0.93296   -1.2629     0.074417  -0.061837   0.9251    -0.25308
 -2.183     -1.3639    -0.30995   -0.98977    1.6252    -1.0291
 -0.047819   0.64689    0.062647   0.54722   -0.36114    0.15535
  1.0872   ]


In [ ]:
print(w2v_model.most_similar(positive=["potato", "burger"]))

[('chicken', 0.9732475280761719), ('cheese', 0.9607888460159302), ('waffle', 0.9480124115943909), ('salad', 0.943723201751709), ('fried', 0.9288325905799866), ('steak', 0.926487147808075), ('pepper', 0.9241527915000916), ('fries', 0.9237129092216492), ('soup', 0.9213549494743347), ('spicy', 0.9193410873413086)]


In [ ]:
print(w2v_model.most_similar(positive=["potato", "tomato"]))

[('pepper', 0.9478217959403992), ('garlic', 0.9469695091247559), ('avocado', 0.9457836151123047), ('spicy', 0.9435321092605591), ('chicken', 0.9432903528213501), ('beans', 0.9419039487838745), ('onion', 0.9399765729904175), ('fried', 0.9379871487617493), ('cheese', 0.9373629689216614), ('salad', 0.9371017217636108)]


### Реализуйте самостояительно методы $3CosAdd$ и $3CosMul$:

$b^* = \arg \max_{w \in W} \cos (w, a^* - a + b)$

$b^* = \arg \max_{w \in W} \frac{\cos(w, b) \times \cos(w, a^*)}{\cos(w, a)}$

In [ ]:
def three_cos_add(a: str, b: str, a_star: str) -> str:
    # YOUR_CODE_HERE
    """
    функция 3CosAdd для поиска слова b*

    :param a: исходное слово a
    :param b: слово, которое должно быть аналогом
    :param a_star: слово, которое задает изменение, подобное a -> a_star
    :return: слово b*, которое наиболее похоже на b по аналогии
    """
    # Получаем векторы для слов
    a_vec = w2v_model[a]
    b_vec = w2v_model[b]
    a_star_vec = w2v_model[a_star]

    # Вычисляем вектор b_star_vec = a* - a + b
    b_star_vec = a_star_vec - a_vec + b_vec

    # Находим самое близкое слово к вектору b_star_vec
    # Используем метод most_similar по вектору, чтобы найти ближайшее слово
    b_star, _ = w2v_model.most_similar(positive=[b_star_vec], topn=1)[0]

    return b_star

In [ ]:
import numpy as np

def three_cos_mul(a: str, b: str, a_star: str) -> str:
    # YOUR_CODE_HERE
    """
    функция 3CosMul для поиска слова b*.

    :param a: исходное слово a
    :param b: слово, которое должно быть аналогом
    :param a_star: слово, которое задает изменение, подобное a -> a_star
    :return: слово b*, которое наиболее похоже на b по аналогии
    """
    # Получаем векторы для слов
    a_vec = w2v_model[a]
    b_vec = w2v_model[b]
    a_star_vec = w2v_model[a_star]

    # Инициализируем переменные для поиска максимума
    max_score = -np.inf
    b_star = None

    # Проходим по всем словам в словаре
    for word in w2v_model.index_to_key:
        w_vec = w2v_model[word]

        # Вычисляем косинусное сходство для каждого компонента
        cos_w_b = np.dot(w_vec, b_vec) / (np.linalg.norm(w_vec) * np.linalg.norm(b_vec))
        cos_w_a_star = np.dot(w_vec, a_star_vec) / (np.linalg.norm(w_vec) * np.linalg.norm(a_star_vec))
        cos_w_a = np.dot(w_vec, a_vec) / (np.linalg.norm(w_vec) * np.linalg.norm(a_vec))

        # Вычисляем результат по формуле
        # Добавляем маленькое число, чтобы избежать деления на ноль
        score = (cos_w_b * cos_w_a_star) / (cos_w_a + 1e-8)

        # Обновляем, если нашли большее значение
        if score > max_score:
            max_score = score
            b_star = word

    return b_star

In [ ]:
three_cos_add("man", "woman", "king")

'meets'

In [ ]:
three_cos_mul("man", "woman", "king")

'pinkett-smith'

## Эмбеддинги и RNN в PyTorch (5 баллов)

Для использования эмбеддингов в нейросетях на torch нужен специальный слой Embedding.
Посмотрим, как он работает.

In [ ]:
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

In [ ]:
emd_dim = 6
vocab_size = len(vocab.word2idx.keys())
embedding_layer = nn.Embedding(num_embeddings=vocab_size, embedding_dim=emd_dim)

batch_size = 2
seq_len = 5

input_ids = torch.tensor([[1, 2, 3, 4, 5], [1, 5, 6, 0, 0]]).long()
key_padding_mask = input_ids > 0

assert input_ids.shape == (batch_size, seq_len)

embeddings = embedding_layer(input_ids)
print(embeddings)
print(embeddings.shape)
assert embeddings.shape == (batch_size, seq_len, emd_dim)

tensor([[[ 0.5043, -0.7522, -0.6435, -0.1807,  0.3319,  0.0331],
         [-0.1590,  0.4813,  1.2450, -0.5982,  0.5961, -0.8716],
         [ 0.5868,  0.7576, -1.0532,  0.5543, -0.3092, -1.8005],
         [ 0.0556, -0.1065,  2.4490, -0.7764, -0.0313,  2.0412],
         [ 1.3848,  0.0979,  1.5388,  1.3377, -0.1576,  1.8117]],

        [[ 0.5043, -0.7522, -0.6435, -0.1807,  0.3319,  0.0331],
         [ 1.3848,  0.0979,  1.5388,  1.3377, -0.1576,  1.8117],
         [-0.2404,  0.1972,  0.5356, -0.1439, -0.6235, -0.3895],
         [ 0.1295, -0.1989, -0.0355,  1.8305,  1.8978, -0.9902],
         [ 0.1295, -0.1989, -0.0355,  1.8305,  1.8978, -0.9902]]],
       grad_fn=<EmbeddingBackward0>)
torch.Size([2, 5, 6])


В качестве RNN возьмем двунаправленную LSTM.

In [ ]:
hidden_size = 7
n_layers = 3

rnn = nn.LSTM(
    input_size=emd_dim,          # Should match embedding dimension
    hidden_size=hidden_size,     # Hidden state size
    num_layers=n_layers,         # Number of layers
    bidirectional=True,          # Make it bidirectional
    batch_first=True             # Batch dimension comes first
)

# Прочитайте туториал по паддингу в RNN
# https://www.geeksforgeeks.org/how-do-you-handle-sequence-padding-and-packing-in-pytorch-for-rnns/


# Вычислем фактическую длину последовательности (количество не дополненных элементов)
input_lengths = key_padding_mask.sum(dim=1)

# Упаковывыем дополненную последовательность
packed_input = torch.nn.utils.rnn.pack_padded_sequence(
    embeddings,
    lengths=input_lengths, # Calculates the lengths of each sequence by summing the True values in key_padding_mask.
    batch_first=True,
    enforce_sorted=False   # Allows sequences to be unsorted; if True, sequences must be sorted by length in descending order.
)
rnn_outputs, (h, c) = rnn(packed_input)

# Преобразует упакованную последовательность обратно в дополненную последовательность.
rnn_outputs, input_sizes = torch.nn.utils.rnn.pad_packed_sequence(
    rnn_outputs,
    batch_first=True,
    total_length=seq_len
)
assert rnn_outputs.shape == (batch_size, seq_len, hidden_size * 2)

🤔 Почему для рекуррентных сетей нужен специальный паддинг?

In [ ]:
"""
Чтобы все последовательности имели одинаковую длину (для обработки в виде одного батча),
нужно добавить "паддинг" (обычно нули) в более короткие последовательности,
что позволяет обработать их вместе с более длинными последовательностями.

Если паддинг (например, нули) просто добавляется к более коротким последовательностям,
рекуррентная сеть будет обрабатывать эти паддинговые токены наравне с полезными данными.
Это может привести к обучению сети на бесполезных данных и ухудшению качества модели,
так как паддинговые токены не несут смысла, но все равно будут влиять на состояние сети.

Поэтому для корректной работы с последовательностями переменной длины применяются специальные механизмы для обработки паддинга.

"""

## Извлечение именованных сущностей (NER) (20 баллов)

Теперь мы готовы к тому, чтобы создать полную сеть для решения задачи распознавания именованных сущностей через потокенную классификацию.

In [ ]:
from torch.utils.data import DataLoader
from tqdm.auto import tqdm

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

### Архитектура и подготовка обучения (10 баллов)

In [ ]:
class TokenClassificationModel(nn.Module):
    def __init__(self, vocab_size: int, embedding_dim: int, hidden_size: int, n_layers: int, n_classes: int):
        super().__init__()
        self.embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim)
        self.rnn = nn.LSTM(
            input_size=embedding_dim,
            hidden_size=hidden_size,
            num_layers=n_layers,
            bidirectional=True,
            batch_first=True
        )
        self.fc = nn.Linear(hidden_size * 2, n_classes)  # Bidirectional
        self.n_classes = n_classes  # Ensure n_classes is defined
        self.loss_fn = nn.CrossEntropyLoss(ignore_index=-100, reduction="none")

    def forward(self, x: torch.Tensor, mask: torch.Tensor):
        # YOUR_CODE_HERE
        # Move data to the same device as the model
        device = next(self.parameters()).device
        embeddings = self.embedding(x.to(device))
        input_lengths = mask.sum(dim=1)
        packed_input = torch.nn.utils.rnn.pack_padded_sequence(
            embeddings,
            lengths=input_lengths,
            batch_first=True,
            enforce_sorted=False
        )

        packed_rnn_output, _ = self.rnn(packed_input.to(device))
        rnn_output, _ = torch.nn.utils.rnn.pad_packed_sequence(
            packed_rnn_output, batch_first=True, total_length=x.size(1)
        )

        logits = self.fc(rnn_output)
        return logits

    def training_step(self, batch):
        # YOUR_CODE_HERE
        # Move data to the same device as the model
        device = next(self.parameters()).device
        input_ids = batch["input_ids"]
        mask = batch["mask"]
        labels = batch["labels"]

        # Check if input indices are within bounds
        assert input_ids.max().item() < self.embedding.num_embeddings, "Token index out of bounds!"

        logits = self(input_ids, mask)
        logits = logits.view(-1, self.n_classes)  # Using the n_classes attribute
        labels = labels.view(-1).to(device)
        loss = self.loss_fn(logits, labels).mean()
        return loss

    def validation_step(self, batch):
       # YOUR_CODE_HERE
        # Move data to the correct device
        device = next(self.parameters()).device
        input_ids = batch["input_ids"]
        mask = batch["mask"]
        labels = batch["labels"]

        # Forward pass
        logits = self(input_ids, mask)

        # Reshape logits to [batch_size * seq_len, n_classes]
        logits = logits.view(-1, self.n_classes)

        # Reshape labels to [batch_size * seq_len]
        labels = labels.view(-1).to(device)

        # Calculate loss
        loss = self.loss_fn(logits, labels).mean()
        return loss

    def configure_optimizers(self):
        # YOUR_CODE_HERE
        return torch.optim.Adam(self.parameters(), lr=1e-3)



In [ ]:
def collate_fn(data):
    input_ids = []
    masks = []
    labels = []
    max_len = 0

    for sample in data:
        # Преобразовываем токены в соответствующие им индексы в словаре
        input_ids.append([vocab.encode_word(token) for token in sample["tokens"]])
        # Преобразование меток в индексы с помощью label2idx
        labels.append([label2idx[y] for y in sample["labels"]])
        # Маскируем 1 для фактических токенов и 0 для падинглв.
        l = len(input_ids[-1])
        masks.append([1] * l)
        # Отслеживайте максимальную длину последовательности в пакете.
        max_len = max(max_len, l)

    # Дополнение последовательностей до максимальной длины в пакете
    padded_input_ids = [ids + [0] * (max_len - len(ids)) for ids in input_ids]
    # -100 для игнорирования меток заполнения
    padded_labels = [lbls + [-100] * (max_len - len(lbls)) for lbls in labels]
    padded_masks = [m + [0] * (max_len - len(m)) for m in masks]

    # Преобразуем  в тензоры
    batch = {
        "input_ids": torch.tensor(padded_input_ids, dtype=torch.long),
        "mask": torch.tensor(padded_masks, dtype=torch.long),
        "labels": torch.tensor(padded_labels, dtype=torch.long)
    }

    return batch


In [ ]:
# Sample model parameters
vocab_size = 10
embedding_dim = 6
hidden_size = 7
n_layers = 2
n_classes = 5

# for testing
train_batch = {
    "input_ids": torch.tensor([[1, 2, 3, 4, 5], [1, 5, 6, 0, 0]]),
    "mask": torch.tensor([[1, 1, 1, 1, 1], [1, 1, 1, 0, 0]]),
    "labels": torch.tensor([[0, 1, 2, 3, 4], [0, 4, 3, -100, -100]])
}

# инициализируем модель
model = TokenClassificationModel(vocab_size, embedding_dim, hidden_size, n_layers, n_classes)

# Запуск прямого проход, чтобы проверить выходные данные модели.
logits = model(train_batch["input_ids"], train_batch["mask"])

# Проверяем логиты
logits, logits.shape


(tensor([[[-0.0561,  0.1584, -0.2384,  0.2395,  0.0246],
          [-0.0492,  0.1885, -0.2192,  0.2454, -0.0037],
          [-0.0491,  0.1980, -0.2127,  0.2378, -0.0022],
          [-0.0516,  0.1868, -0.2082,  0.2279,  0.0077],
          [-0.0551,  0.2256, -0.1790,  0.2557, -0.0051]],
 
         [[-0.0627,  0.1702, -0.2157,  0.2443,  0.0239],
          [-0.0698,  0.2075, -0.1848,  0.2622,  0.0027],
          [-0.0969,  0.2036, -0.1730,  0.2497,  0.0159],
          [-0.0554,  0.1649, -0.2087,  0.2566,  0.0655],
          [-0.0554,  0.1649, -0.2087,  0.2566,  0.0655]]],
        grad_fn=<ViewBackward0>),
 torch.Size([2, 5, 5]))

In [ ]:
optimizer = model.configure_optimizers()

model.train()
optimizer.zero_grad()
loss = model.training_step(train_batch)
loss.backward()
optimizer.step()

# Output the loss after one step
loss.item()


1.2940717935562134

In [ ]:
idx2label = ["O", "B-MISC", "I-MISC", "B-PER", "I-PER", "B-ORG", "I-ORG", "B-LOC", "I-LOC"]
label2idx = {label: i for i, label in enumerate(idx2label)}

In [ ]:
def read_data(path: str) -> List[Dict[str, List[str]]]:
    samples = []
    with open(path) as f:
        sentences = f.read().split("\n\n")
    for sentence in sentences:
        if "-DOCSTART-" in sentence:
            continue
        tokens = []
        labels = []
        for line in sentence.strip().split("\n"):
            if not line:
                continue
            line = line.split()
            tokens.append(line[0])
            labels.append(line[-1])
        if not tokens:
            continue
        samples.append({"tokens": tokens, "labels": labels})
    return samples

# load data

In [ ]:
import gdown

In [ ]:
file_id = "1uJvjwH9_Mlau38bJoump2LZ3LiP_K1wI"
file_name = "conll2003.zip"
gdown.download('https://drive.google.com/uc?id=' + file_id, file_name, quiet=False)

!unzip conll2003.zip -d data

Downloading...
From: https://drive.google.com/uc?id=1uJvjwH9_Mlau38bJoump2LZ3LiP_K1wI
To: /content/conll2003.zip
100%|██████████| 959k/959k [00:00<00:00, 121MB/s]

Archive:  conll2003.zip
   creating: data/conll2003/
  inflating: data/conll2003/train.txt  
  inflating: data/conll2003/valid.txt  
  inflating: data/conll2003/test.txt  
  inflating: data/conll2003/metadata  


In [ ]:
train_data = read_data("./data/conll2003/train.txt")
val_data = read_data("./data/conll2003/valid.txt")
test_data = read_data("./data/conll2003/test.txt")
print(len(train_data), len(val_data), len(test_data))

14041 3250 3453


In [ ]:
train_data[1]

{'tokens': ['Peter', 'Blackburn'], 'labels': ['B-PER', 'I-PER']}

Прочитайте про схемы тегирования в NER: https://en.wikipedia.org/wiki/Inside–outside–beginning_(tagging)

🤔 Какая схема тегирования используется в этом датасете?

In [ ]:
N_EPOCHS = 3 #YOUR_CODE_HERE
BATCH_SIZE = 10 #YOUR_CODE_HERE

In [ ]:
train_dl = DataLoader(train_data, collate_fn=collate_fn, batch_size=BATCH_SIZE, shuffle=True)
val_dl = DataLoader(val_data, collate_fn=collate_fn, batch_size=BATCH_SIZE, shuffle=False)
test_dl = DataLoader(test_data, collate_fn=collate_fn, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
dataloader_iterator = iter(val_dl)
next(dataloader_iterator)

{'input_ids': tensor([[   78,    56,  1286,   157,    89,  1189, 10158,  1443,     3,   292,
           2847,  1422,   136,  1805,  1201,    20,     3,    68,   969,    69,
             75,   200,   152,     3,    11,     0,     0,     0,     0,     0,
              0,     0,     0,     0],
         [ 6504,   747,   284,    16,  4453,    68,   186,   581,  1743,  1713,
            678,     3,    89,     3,  1812,     7,  5067,    68,  1626,   217,
            186,   454,  1823,    69,   632,   495,  2547,  1590,    50,     3,
            148,  1823,  3141,    11],
         [  305,  1480,   522,   681,   882,    68,  2444,   128,  1580,  2357,
             69,     3,    69,    82,  1445,     3,    69,  4793,    69,   681,
           6504,     7,     3,    82,    75,   581,  1823,   200,   152,     3,
             11,     0,     0,     0],
         [  113,    16,     3,   152,  4622,    16,   346,  1257,   152,   186,
           1531,  8198,  4844,    89,    16,  4714,    69,  1774,  511

In [ ]:
vocab = Vocabulary([" ".join(example["tokens"]) for example in train_data], min_count=2, lower=True)

In [ ]:
vocab_size = len(vocab)
vocab_size

10946

In [ ]:
n_classes = len(idx2label)
ner_model =  TokenClassificationModel(vocab_size, embedding_dim, hidden_size, n_layers, n_classes)

In [ ]:
# инициализируйте эмбеддинги модели через эмбеддинги word2vec
# YOUR_CODE_HERE

# фунуция создания матрицы через эмбеддинги word2vec
def create_embedding_matrix(vocab, embedding_model, embedding_dim):
    vocab_size = len(vocab.word2idx)
    embedding_matrix = np.zeros((vocab_size, embedding_dim))  # инициализируем нуляими

    for word, idx in vocab.word2idx.items():
        if word in embedding_model:
            embedding_vector = embedding_model[word]
            # вставляем предтренированный вектор в строку матрицы
            embedding_matrix[idx] = embedding_vector
        else:
            # получаем случайнык вектора из нормального распределения для OOV слов
            embedding_matrix[idx] = np.random.normal(size=(embedding_dim,))

    return torch.tensor(embedding_matrix, dtype=torch.float32)

# инициализируем матрицу для словаря
embedding_dim = 25  # Dimension of GloVe embeddings
embedding_matrix = create_embedding_matrix(vocab, w2v_model, embedding_dim)

# заморозьте слой эмбеддингов
# YOUR_CODE_HERE

# инициализируем слой модели
embedding_layer = nn.Embedding.from_pretrained(embedding_matrix, freeze=False)  # By default, freeze=False

# можно заморозить слой во время обучения.
embedding_layer.weight.requires_grad = False  # Freeze the embeddings

print(embedding_layer)

Embedding(10946, 25)


🤔 Как происходит инициализация эмбеддингов для токенов, отсутствующих в модели word2vec?

In [ ]:
"""
1. Можно присвоить случайные вектора из распределения.
2. Можно через инициадизацию от токена <UNK> незвестного слова
3. Можно брать из других корпусов
4. можно посмотреть по окружению слов и найти близкое по контексту

"""

### Обучение и инференс (10 баллов)

In [ ]:
ner_model.to(device)
print(ner_model)

TokenClassificationModel(
  (embedding): Embedding(10946, 6)
  (rnn): LSTM(6, 7, num_layers=2, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=14, out_features=9, bias=True)
  (loss_fn): CrossEntropyLoss()
)


In [ ]:
optimizer = ner_model.configure_optimizers()

In [ ]:
from tqdm import tqdm

N_EPOCHS = 15  # количество эпох

for epoch in range(N_EPOCHS):
    ner_model.train()
    train_losses = []
    val_losses = []

    # Training loop
    for i, batch in tqdm(enumerate(train_dl), total=len(train_dl)):
        optimizer.zero_grad()  # Clear previous gradients

        # Forward pass and loss calculation
        loss = ner_model.training_step(batch)
        train_losses.append(loss.detach().cpu().numpy())

        # Backpropagation
        loss.backward()

        # Optimizer step
        optimizer.step()

    # Validation loop
    ner_model.eval()  # Set model to evaluation mode
    with torch.no_grad():  # No gradient calculation for validation
        for i, batch in tqdm(enumerate(val_dl), total=len(val_dl)):
            loss = ner_model.validation_step(batch)
            val_losses.append(loss.cpu().numpy())

    # Calculate average losses for the epoch
    train_loss = sum(train_losses) / len(train_losses)
    val_loss = sum(val_losses) / len(val_losses)

    # Print training and validation losses for the current epoch
    print(f"{epoch = }: {train_loss = }, {val_loss = }")

    # Save the model state after each epoch
    savepath = f"ner_model_{epoch}ep.bin"
    torch.save(ner_model.state_dict(), savepath)


100%|██████████| 325/325 [00:01<00:00, 277.98it/s]


epoch = 0: train_loss = 0.13343377133095605, val_loss = 0.1896311459174523


100%|██████████| 325/325 [00:00<00:00, 371.88it/s]


epoch = 1: train_loss = 0.11862591085060635, val_loss = 0.17554832592033423


100%|██████████| 325/325 [00:00<00:00, 385.90it/s]


...

epoch = 12: train_loss = 0.04844296150923677, val_loss = 0.11212174191282918


100%|██████████| 325/325 [00:00<00:00, 391.92it/s]


epoch = 13: train_loss = 0.04532344390131082, val_loss = 0.11068233816454617


100%|██████████| 325/325 [00:00<00:00, 379.06it/s]


epoch = 14: train_loss = 0.042644473054244104, val_loss = 0.10729621725586744


In [ ]:
def predict(model, test_data) -> List[List[str]]:
    with torch.no_grad():
        model.eval()
        predictions = []

        # YOUR_CODE_HERE
        ...
        return predictions  # предсказанные теги

In [ ]:
def predict(model, test_dl) -> List[List[str]]:
    model.eval()  # Set model to evaluation mode
    predictions = []
    # YOUR_CODE_HERE

    # Сопоставление индекса с тегом (обратно label2idx)
    idx2label = {v: k for k, v in label2idx.items()}

    with torch.no_grad():  # Отключаем вычисление градиента для прогнозирования
        for batch in test_dl:
            input_ids = batch["input_ids"]
            mask = batch["mask"]

            # прямой проход
            logits = model(input_ids, mask).to(device)

            # Получаем прогнозируемые индексы классов (через argmax по последнему измерению)
            pred_indices = torch.argmax(logits, dim=-1)

            # Преобразуеи прогнозируемые индексы в метки и добавим в список прогнозов
            for preds, m in zip(pred_indices, mask):
                # Сохраняем прогнозы только для не дополненных токенов (mask = 1)
                pred_tags = [idx2label[idx.item()] for idx, mask_val in zip(preds, m) if mask_val == 1]
                predictions.append(pred_tags)

    return predictions  # предсказанные теги


In [ ]:
predictions = predict(ner_model, test_dl)
for sample, pred in zip(test_data, predictions):
    text = " ".join(sample["tokens"])
    print("*" * 100)
    print(text)
    for i, tag in enumerate(pred):
        print(f"{tag}: {sample['tokens'][i]}")


Выходные данные были обрезаны до нескольких последних строк (5000).
O: 32
****************************************************************************************************
Barnet 22 8 8 6 23 17 32
B-ORG: Barnet
O: 22
O: 8
O: 8
O: 6
O: 23
O: 17
O: 32
****************************************************************************************************
Colchester 22 7 10 5 32 26 31
B-ORG: Colchester
O: 22
O: 7
O: 10
O: 5
O: 32
O: 26
O: 31
****************************************************************************************************
Scunthorpe 22 9 4 9 28 30 31
B-ORG: Scunthorpe
O: 22
O: 9
O: 4
O: 9
O: 28
O: 30
O: 31
****************************************************************************************************
Northampton 22 8 6 8 31 26 30
B-ORG: Northampton
O: 22
O: 8
O: 6
O: 8
O: 31
O: 26
O: 30
****************************************************************************************************
Scarborough 21 7 9 5 30 27 30
B-ORG: Scarborough
O: 21
O: 7
O: 9
O: 5
O: 30
O: 27

Используя библиотеку https://github.com/chakki-works/seqeval, рассчитайте метрики.

Результаты SotA-решений вы найдете на странице https://paperswithcode.com/sota/named-entity-recognition-ner-on-conll-2003.

In [ ]:
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=7b6982665f5f86e9a349186c4dca4a1d1c47dbbb430c39cef6bf33944b6812df
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [ ]:
from seqeval.metrics import classification_report

# данные из датасета test_data
true_labels = [sample['labels'] for sample in test_data]
# предсказанные метки моделью
predicted_labels = predictions

# YOUR_CODE_HERE
report = classification_report(true_labels, predicted_labels)
# Calculate and print the classification report
print(report)


              precision    recall  f1-score   support

         LOC       0.58      0.46      0.51      1668
        MISC       0.31      0.10      0.15       702
         ORG       0.27      0.22      0.24      1661
         PER       0.44      0.38      0.41      1617

   micro avg       0.42      0.32      0.36      5648
   macro avg       0.40      0.29      0.33      5648
weighted avg       0.41      0.32      0.36      5648



### Дополнительное задание (не оценивается)

(Опционально) Попробуйте повысить метрики. Вот несколько идей:

1. Взять более тяжелые эмбеддинги.
2. На замораживать слой эмбеддингов.
3. Использовать MLP после RNN.
4. Умная инициализация весов, дропаут, продвинутый оптимизатор, скедулер и т. д.

In [ ]:
class TokenClassificationModel_new(TokenClassificationModel):  # Наследуем от TokenClassificationModel
    def __init__(self, vocab_size: int, embedding_dim: int, hidden_size: int, n_layers: int, n_classes: int, embedding_matrix=None):
        super().__init__(vocab_size=vocab_size, embedding_dim=embedding_dim, hidden_size=hidden_size, n_layers=n_layers, n_classes=n_classes) # Наследуем всё из TokenClassificationModel

        # Если переданы предобученные эмбеддинги, то инициализируем их
        if embedding_matrix is not None:
            # Извлекаем размерность эмбеддингов из переданной матрицы
            embedding_dim = embedding_matrix.shape[1]
            # Используем предобученные эмбеддинги с возможностью их дообучения (freeze=False)
            self.embedding = nn.Embedding.from_pretrained(embedding_matrix, freeze=False)
        else:
            # Если предобученные эмбеддинги не переданы, инициализируем их случайно
            self.embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim)

        # LSTM с bidirectional=True
        self.rnn = nn.LSTM(
            input_size=embedding_dim,
            hidden_size=hidden_size,
            num_layers=n_layers,
            bidirectional=True,
            batch_first=True
        )
        # Полносвязный слой, сопоставляющий выход RNN с классами
        self.fc = nn.Linear(hidden_size * 2, n_classes)  # Умножаем hidden_size на 2, т.к. LSTM двунаправленная

        # Количество классов и функция потерь
        self.n_classes = n_classes
        self.loss_fn = nn.CrossEntropyLoss(ignore_index=-100, reduction="none")


In [ ]:
vocab = Vocabulary([" ".join(example["tokens"]) for example in train_data], min_count=2, lower=True)
enb_dim = 25
# Create the embedding matrix using pre-trained GloVe embeddings
embedding_matrix = create_embedding_matrix(vocab, w2v_model, embedding_dim=enb_dim)
n_classes = len(idx2label)
# Initialize the model with pre-trained embeddings
ner_model = TokenClassificationModel_new(vocab_size=len(vocab), embedding_dim=enb_dim, hidden_size=7, n_layers=3, n_classes=n_classes, embedding_matrix=embedding_matrix)

In [ ]:
ner_model.to(device)
print(ner_model)

TokenClassificationModel_new(
  (embedding): Embedding(10946, 25)
  (rnn): LSTM(25, 7, num_layers=3, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=14, out_features=9, bias=True)
  (loss_fn): CrossEntropyLoss()
)


In [ ]:
optimizer = ner_model.configure_optimizers()

In [ ]:
from tqdm import tqdm

N_EPOCHS = 15  # Example number of epochs, adjust as needed

for epoch in range(N_EPOCHS):
    ner_model.train()
    train_losses = []
    val_losses = []

    # Training loop
    for i, batch in tqdm(enumerate(train_dl), total=len(train_dl)):
        optimizer.zero_grad()  # Clear previous gradients

        # Forward pass and loss calculation
        loss = ner_model.training_step(batch)
        train_losses.append(loss.detach().cpu().numpy())

        # Backpropagation
        loss.backward()

        # Optimizer step
        optimizer.step()

    # Validation loop
    ner_model.eval()  # Set model to evaluation mode
    with torch.no_grad():  # No gradient calculation for validation
        for i, batch in tqdm(enumerate(val_dl), total=len(val_dl)):
            loss = ner_model.validation_step(batch)
            val_losses.append(loss.cpu().numpy())

    # Calculate average losses for the epoch
    train_loss = sum(train_losses) / len(train_losses)
    val_loss = sum(val_losses) / len(val_losses)

    # Print training and validation losses for the current epoch
    print(f"{epoch = }: {train_loss = }, {val_loss = }")

    # Save the model state after each epoch
    savepath = f"ner_model_{epoch}ep.bin"
    torch.save(ner_model.state_dict(), savepath)


100%|██████████| 325/325 [00:01<00:00, 309.45it/s]


epoch = 0: train_loss = 0.2585353764138612, val_loss = 0.2718985028450306


100%|██████████| 325/325 [00:01<00:00, 297.83it/s]


epoch = 1: train_loss = 0.1348484023066496, val_loss = 0.16420096250680777


100%|██████████| 325/325 [00:01<00:00, 310.56it/s]


...

epoch = 12: train_loss = 0.01781141842977273, val_loss = 0.10502100279733825


100%|██████████| 325/325 [00:01<00:00, 310.74it/s]


epoch = 13: train_loss = 0.015871861450054293, val_loss = 0.11039704307251108


100%|██████████| 325/325 [00:01<00:00, 308.53it/s]

epoch = 14: train_loss = 0.01426209645019347, val_loss = 0.11407187999584353


In [ ]:
predictions = predict(ner_model, test_dl)

# Assuming `predictions` are your predicted tags and `test_data` has the true tags
true_labels = [sample['labels'] for sample in test_data]  # True labels from the dataset
predicted_labels = predictions  # Predicted labels from the model

# YOUR_CODE_HERE
report = classification_report(true_labels, predicted_labels)
# Calculate and print the classification report
print(report)


              precision    recall  f1-score   support

         LOC       0.75      0.79      0.77      1668
        MISC       0.64      0.58      0.61       702
         ORG       0.63      0.58      0.60      1661
         PER       0.70      0.62      0.66      1617

   micro avg       0.69      0.65      0.67      5648
   macro avg       0.68      0.64      0.66      5648
weighted avg       0.69      0.65      0.67      5648

